In [1]:
5+5

10

In [2]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyMuPDFLoader,DirectoryLoader, PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers

c:\Users\user\anaconda3\envs\medical_chatbot_opensource_llm_llama2\lib\site-packages\pinecone\index.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [ ]:
# embedding model in dimension is 384, this will be useful while creating PINECONE_API_ENV

PINECONE_API_KEY = 'XXXXXXXXXXXXXX'
PINECONE_API_ENV = 'XXXXXX' 

embedding_model_name="sentence-transformers/all-MiniLM-L6-v2"


In [ ]:
#extract the data from pdf

def load_pdf(data):
   loader =  DirectoryLoader(data,
                    glob = '*.pdf',
                    loader_cls=PyPDFLoader)
   documents = loader.load()

   return documents


In [ ]:
extracted_data = load_pdf('data/')

In [ ]:
type(extracted_data)

In [ ]:
#page wise data

extracted_data[2].page_content

In [ ]:
# create text chunks

def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500 , chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks




In [ ]:
text_chunks = text_split(extracted_data)
print('length of text chunks :', len(text_chunks))

In [ ]:
type(text_chunks)

In [ ]:
# download embedding model

def download_hugging_face_embeddings(embedding_model_name):
    embeddings = HuggingFaceEmbeddings(model_name=embedding_model_name)
    return embeddings



In [ ]:
embeddings = download_hugging_face_embeddings(embedding_model_name)

In [ ]:
embeddings

In [ ]:
query_result = embeddings.embed_query('Hello world')
print('Length : ', len(query_result))

In [ ]:
query_result


In [ ]:
#Initializing the Pinecone
pinecone.init(api_key=PINECONE_API_KEY,
              environment=PINECONE_API_ENV)

index_name="medical-chatbot-opensource-llm-llama2"

#Creating Embeddings for Each of The Text Chunks & storing
docsearch=Pinecone.from_texts([t.page_content for t in text_chunks], embeddings, index_name=index_name)

In [ ]:
# if we already have an index, we can load it like this.
pinecone.init(api_key=PINECONE_API_KEY,
              environment=PINECONE_API_ENV)

index_name="medical-chatbot-opensource-llm-llama2"

docsearch = Pinecone.from_existing_index(index_name,embeddings)

query = "what are allergies ?"

docs = docsearch.similarity_search(query, k=3)

print("Result : ", docs)

In [ ]:
# send these to LLM ang get the correct answer

prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [ ]:
PROMPT = PromptTemplate(template=prompt_template, input_variables=["context" , "question"])
chain_type_kwargs={"prompt": PROMPT}

In [ ]:
llm = CTransformers(model = "model/llama-2-7b-chat.ggmlv3.q4_0.bin", model_type="llama",
                    config = {'max_new_tokens':512,
                          'temperature':0.8})


In [ ]:
# create question answering object

qa=RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=docsearch.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)

In [ ]:
user_input=input(f"Input Prompt:")
result=qa({"query": user_input})
print("Response : ", result["result"])

In [ ]:
# ask a question

# while True:
#     user_input=input(f"Input Prompt:")
#     result=qa({"query": user_input})
#     print("Response : ", result["result"])